# DO NOT RUN THIS NOTEBOOK--READS.TXT NOT AVAILABLE

**Q1**. Finding sequencing errors.

Q1. Finding sequencing errors.

We generated a simulated set of sequenced DNA reads of the 16 yeast chromosomes at 30-fold coverage in the file data/HW_KMER/reads.txt. Each read has a length of 75 bases. Some of the reads have sequencing errors. We would like to identify which lines in the file contains reads with sequencing errors.

(Part 1) One simple way to do this is by k-mer (a DNA sequence of length k) counting. Consider the k-mers generated by a shifting window of width k for each read. If the sequence has no error, each k-mer will likely be found in multiple copies due to the 30-fold coverage. However, if there is a random sequencing error, the k-mer will likely be found in only 1 or a few copies (depends on length of k). Use k=15. Identify the lines likely to contain reads with sequencing errors as any line containing a k-mer with a count of only 1. Because we are dealing with a potentially large number of reads, write the code so that it minimizes memory usage using streams. You may use the toolz library if you think it will be helpful. (50 points)

(Part 2) The actual line numbers for reads with sequencing errors is found in 'data/HW_KMER/error_lines.txt. Line numbers start with 0, not 1. Of the reads that you believe to have errors, how many are false positives? For each false positive read, identify the chromosome(s) and starting position(s) it comes from. (50 points)

Chromosome sequences in FASTA format are in

```
data/HW_KMER/chr01.fas
data/HW_KMER/chr02.fas
data/HW_KMER/chr03.fas
data/HW_KMER/chr04.fas
data/HW_KMER/chr05.fas
data/HW_KMER/chr06.fas
data/HW_KMER/chr07.fas
data/HW_KMER/chr08.fas
data/HW_KMER/chr09.fas
data/HW_KMER/chr10.fas
data/HW_KMER/chr11.fas
data/HW_KMER/chr12.fas
data/HW_KMER/chr13.fas
data/HW_KMER/chr14.fas
data/HW_KMER/chr15.fas
data/HW_KMER/chr16.fas
```

**Hints**: 

1. Check your code on a smaller version of the original data or your own simulated data before applying it to the full data set (which can take a long time).
2. You only need to consider k-mers from sliding windows generated for a single line. Ignore k-mers that wrap across two lines.
3. When generating k-mers, also keep a record of the line number that the k-mer was generated from. This avoids having to later search all reads to find out where a k-mer comes from, which is a very expensive operation.

# Part 1

In [1]:
import itertools as it

In [11]:
path = 'data/HW_KMER/reads.txt'

def getReads(path):
    """getReads will generate every read in reads.txt"""
    reads_data = open(path, 'r')
    for read in reads_data:
        yield read.strip()
    reads_data.close()

In [3]:
def addReadToDict(readSequence, myDict, lineNumber):
    """addReadsToDict will divide a read into multiple 15-mers and add the 15-mer into a dictionary"""
    start = 0
    end = 15
    for read in readSequence:
        newRead = readSequence[start:end]
        myDict.setdefault(newRead, [])
        myDict[newRead].append(lineNumber)
        start = start + 1
        end = end + 1
        if(end > 74):
            break
    return myDict

In [4]:
kmer_list = {}
i = 0

for read in getReads(path):
    addReadToDict(read, kmer_list, i)
    i = i+1
    if (i>4400000): # Note: Had to break code here since kernel crashes without the break statement
        break

In [5]:
import sys
sys.getsizeof(kmer_list)

402653280

In [6]:
sequence_errors = {v[0] for k, v in kmer_list.items() if len(v) == 1} # List of line numbers corresponding to reads w/ errors

# Part 2

In [12]:
path2 = 'data/HW_KMER/error_lines.txt'

In [9]:
errors = []
trueErrors = open(path2, 'r')
for error in trueErrors:
    errors.append(int(error.strip()))
errorsSet = set(errors)

In [13]:
falsePositives = sequence_errors - errorsSet
sequence_errors - errorsSet

{2326716, 4237450}

**There are two false-positives at read #'s 2,326,716 and 4,237,450.**

In [37]:
j = 0
target1 = ""
target2 = ""
for read in getReads(path):
    if(j == 2326716):
        target1 = read
        j = j+1
    elif(j == 4237450):
        target2 = read
        j = j+1
    elif(j > 4237450):
        break
    else:
        j = j+1

In [85]:
print("The first false positive is the sequence: ", target1)

The first false positive is the sequence:  GGTAGTGTGGTGTGTGGGTGTGGGTGTGGATGTGGTGTGGATGTGGTGTGGGTGTGGATGTGGGTGTGGTGTGTG


In [86]:
print("The second false positive is the sequence: ", target2)

The second false positive is the sequence:  TTTTAATTTCGGTCAGAAAGCCGGGTAAGGTATGACAGCGAGAGTAGAGGTAGATGTGAGAGAGTGTGTGGGTGT


### Finding the sequence

In [55]:
import re

In [84]:
# Search every chromosome to find the false positive reads
for i in range(1,17):
    with open('data/HW_KMER/chr%02d.fas'%i) as f:
        sequenceText = f.read()
        sequenceText = sequenceText.split('\n', 1)[-1]
        sequenceText = sequenceText.replace('\n', "")
        res1 = re.search(target1, sequenceText)
        res2 = re.search(target2, sequenceText)
        if res1:
            print("False Positive 1 occurs on Chromosome ", i, "at starting index position " , res1.start() + 1)
        if res2:
            print("False Positive 2 occurs on Chromosome ", i, "at starting index position " , res2.start() + 1)

False Positive 1 occurs on Chromosome  9 at starting index position  439811
False Positive 2 occurs on Chromosome  15 at starting index position  1091210
